In [13]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
from pprint import pprint
import requests
import time
key = '5h567tf39qwsksfp9uxakxfc'

## Source

### https://developer.sportradar.com/

- nfl api


## working with json results


- Create a function that inputs:
    - year 
    - season(pre,regular,post)
    - api key

- output:
    - json result

In [11]:

def nfl_schedule(year,season,key):
    '''returns json result from api call'''
    URL = 'https://api.sportradar.us/nfl-ot2/games/'
    path = '{}/{}/schedule.json'.format(year,season)
    payload = {'api_key':key}
    r = requests.get(URL+path,params=payload)
    return r.json()

## Test function

- get data from 2012 regular season

In [22]:
nfl12 = nfl_schedule('2012','REG',key)

In [23]:
nfl12.keys()

dict_keys(['id', 'year', 'type', 'name', 'weeks', '_comment'])

## `nfl12`
|key  | value  |description|
|:---|---|---|
|id   |'1adca71b-202f.'|id for 2012 season  |
|year   | 2012  |duh |
|type   |'REG'   |$duh^2$ |
|name   | 'REG'  |donno. |
|weeks   | list  |list of every week data |
|_comment|'Generation started @..'| # when the api call was ran|

In [68]:
nfl12['id']

'1adca71b-202f-44d5-9fab-abd45d813954'

In [69]:
nfl12['year']

2012

In [70]:
nfl12['type']

'REG'

In [71]:
nfl12['name']

'REG'

In [74]:
len(nfl12['weeks']) # list of week data

17

In [72]:
nfl12['_comment']

'Generation started @ 2018-04-05 03:33:01 +0000 ended @ 2018-04-05 03:33:01 +0000'

In [75]:
nfl12['weeks'][0].keys()

dict_keys(['id', 'sequence', 'title', 'games'])


# First week keys
## `nfl12['weeks'][0]`

|key  |value  |description  |
|---|---|---|
|id  | '2c3721af..'|first week value  |
|sequence  | 1 |?? first week?  |
|title  |1  |same?  |
|games  | list |list of games given the week  |

In [54]:
nfl12['weeks'][0]['id']

'2c3721ac-828a-4ed5-b6be-02b038e7be6f'

In [55]:
nfl12['weeks'][0]['sequence']

1

In [56]:
nfl12['weeks'][0]['title']

'1'

In [28]:
len(nfl12['weeks'][0]['games'])  # 16 games

16

# First week, First game
## `nfl12['weeks'][0]['games'][0]`

|key      | value  |description|
|:-------|------|---|
|id       |`0651e14f-55b0-..`| id for first game of first week  |
|status   |'closed|game done?   |
|reference|`55509`|no idea   |
|number   |6| same  |
|scheduled|'2012-09-09T17:03:08+00:00'| date time of game  |
|attendance|71180|home team stadium attendance   |
|utc_offset|-6| something with time  |
|entry_mode|'INGEST'| no idea  |
|venue     |dict|info on home stadium   |
|home      |dict|info on home team  |
|away      |dict| info on away team  |
|broadcast |dict|info on where it was shown   |
|scoring   |dict|final scores and for each period  |

In [38]:
nfl12['weeks'][0]['games'][0]['id']

'0651e14f-55b0-403f-9ff0-d0f11261490d'

In [39]:
nfl12['weeks'][0]['games'][0]['status']

'closed'

In [41]:
nfl12['weeks'][0]['games'][0]['reference']

'55509'

In [42]:
nfl12['weeks'][0]['games'][0]['number']

6

In [44]:
nfl12['weeks'][0]['games'][0]['scheduled']

'2012-09-09T17:03:08+00:00'

In [45]:
nfl12['weeks'][0]['games'][0]['attendance']

71180

In [46]:
nfl12['weeks'][0]['games'][0]['utc_offset']

-6

In [47]:
nfl12['weeks'][0]['games'][0]['entry_mode']

'INGEST'

In [48]:
nfl12['weeks'][0]['games'][0]['venue']

{'address': 'One Arrowhead Drive',
 'capacity': 79451,
 'city': 'Kansas City',
 'country': 'USA',
 'id': '2ec4c411-dac2-403d-b091-6b6aa4a0a914',
 'name': 'Arrowhead Stadium',
 'roof_type': 'outdoor',
 'state': 'MO',
 'surface': 'turf',
 'zip': '64129'}

In [49]:
nfl12['weeks'][0]['games'][0]['home']

{'alias': 'KC',
 'game_number': 1,
 'id': '6680d28d-d4d2-49f6-aace-5292d3ec02c2',
 'name': 'Kansas City Chiefs'}

In [50]:
nfl12['weeks'][0]['games'][0]['away']

{'alias': 'ATL',
 'game_number': 1,
 'id': 'e6aa13a4-0055-48a9-bc41-be28dc106929',
 'name': 'Atlanta Falcons'}

In [52]:
nfl12['weeks'][0]['games'][0]['broadcast']

{'network': 'FOX', 'satellite': '706'}

In [53]:
nfl12['weeks'][0]['games'][0]['scoring']

{'away_points': 40,
 'home_points': 24,
 'periods': [{'away_points': 10,
   'home_points': 3,
   'id': '60a53db0-7de7-4210-bc55-7d146af3e049',
   'number': 1,
   'period_type': 'quarter',
   'sequence': 1},
  {'away_points': 10,
   'home_points': 14,
   'id': 'ffa039f4-fcdc-44d3-8cd0-ab303c0fad39',
   'number': 2,
   'period_type': 'quarter',
   'sequence': 2},
  {'away_points': 14,
   'home_points': 0,
   'id': '16ca0c95-0437-4f7f-a78e-a82cb2ecc08f',
   'number': 3,
   'period_type': 'quarter',
   'sequence': 3},
  {'away_points': 6,
   'home_points': 7,
   'id': 'eaf149e2-cbf8-4e63-a4c8-ef5c6a55fa4f',
   'number': 4,
   'period_type': 'quarter',
   'sequence': 4}]}

# Convert to dataframe

Now that we know how the data is displayed,
lets create a df from our results

steps:
- create an empty list
- loop over the total weeks in the season
- creates a dataframe from that week
- append the dataframe to our empty list
- pd.concat our list to a new dataframe

In [57]:
dflist =[]
for i in range(len(nfl12['weeks'])):
    data = json_normalize(nfl12['weeks'][i]['games'])  # convert to df
    dflist.append(data)  # append week data
nfl = pd.concat(dflist, axis=0)

In [59]:
nfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 15
Data columns (total 32 columns):
attendance             256 non-null int64
away.alias             256 non-null object
away.game_number       256 non-null int64
away.id                256 non-null object
away.name              256 non-null object
broadcast.network      256 non-null object
broadcast.satellite    234 non-null object
entry_mode             256 non-null object
home.alias             256 non-null object
home.game_number       256 non-null int64
home.id                256 non-null object
home.name              256 non-null object
id                     256 non-null object
number                 256 non-null int64
reference              256 non-null object
scheduled              256 non-null object
scoring.away_points    256 non-null int64
scoring.home_points    256 non-null int64
scoring.periods        256 non-null object
status                 256 non-null object
utc_offset             256 non-null int64


In [76]:
nfl.head()

,attendance,away.alias,away.game_number,away.id,away.name,broadcast.network,broadcast.satellite,entry_mode,home.alias,home.game_number,...,venue.capacity,venue.city,venue.country,venue.id,venue.name,venue.roof_type,venue.state,venue.surface,venue.zip,weather
0,71180,ATL,1,e6aa13a4-0055-48a9-bc41-be28dc106929,Atlanta Falcons,FOX,706,INGEST,KC,1,...,79451,Kansas City,USA,2ec4c411-dac2-403d-b091-6b6aa4a0a914,Arrowhead Stadium,outdoor,MO,turf,64129,NaN
1,70523,SF,1,f0e724b0-4cbf-495a-be47-013907608da9,San Francisco 49ers,FOX,714,INGEST,GB,1,...,80750,Green Bay,USA,5a60dd3a-302c-41c6-ab0f-dd335c1103c2,Lambeau Field,outdoor,WI,turf,54304,NaN
2,76823,PIT,1,cb2f9f1f-ac67-424e-9e72-1475cb0ed398,Pittsburgh Steelers,NBC,NaN,INGEST,DEN,1,...,76125,Denver,USA,6589e61d-ef1e-4e30-91b5-9acd2072b8a0,Sports Authority Field at Mile High,outdoor,CO,turf,80204,NaN
3,62341,IND,1,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9,Indianapolis Colts,CBS,708,INGEST,CHI,1,...,61500,Chicago,USA,d7866605-5ac6-4b3a-90e8-760cc5a26b75,Soldier Field,outdoor,IL,turf,60605,NaN
4,56607,JAC,1,f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de,Jacksonville Jaguars,CBS,710,INGEST,MIN,1,...,64111,Minneapolis,USA,1fede74c-50a2-4036-a1c0-b4d1e53ca98c,Mall of America Field,dome,MN,artificial,55415,NaN


## Save our raw data to  to csv

In [79]:
ls sports_data_raw/NFL

nfl10clean.csv*  nfl10yearsfull.csv*  nfl17yearsfull_raw.csv*


In [80]:
nfl.to_csv('sports_data_raw/nfl_12_raw.csv')

In [81]:
## finish